# Data Mining - Homework 3

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
import pandas as pd

data_set_train = pd.read_csv("/content/gdrive/MyDrive/DataMining/loan_sanction_train.csv")
data_set_test = pd.read_csv("/content/gdrive/MyDrive/DataMining/loan_sanction_test.csv")

data_set_train

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


## Preprocess the data

In [ ]:
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

def apply_imputer_most_frequent(data_set):
  '''Treat the missing values with the "most frequent" imputer method.'''

  imputer = SimpleImputer(strategy='most_frequent')

  return imputer.fit_transform(data_set)

def apply_imputer_knn(data_set):
  '''Treat the missing values with the KNN imputer method.'''

  imputer = KNNImputer()

  return imputer.fit_transform(data_set)

def apply_one_hot_encoder(data_set):
  '''Apply one hot encoding to the data set.'''

  enc = OneHotEncoder()

  return enc.fit_transform(data_set).toarray()

def apply_ordinal_encoder(data_set):
  '''Apply ordinal encoding to the data set.'''

  enc = OrdinalEncoder()

  return enc.fit_transform(data_set)

def apply_standardization(data_set):
  '''Apply standardization to the data set.'''

  scaler = StandardScaler()

  return scaler.fit_transform(data_set)

def preprocess_home_loans_data_set(data_set):
  X_data_set_binary = data_set[['Married', 'Education', 'Self_Employed', 'Credit_History']]
  X_data_set_binary = apply_imputer_most_frequent(X_data_set_binary)
  X_data_set_binary = apply_ordinal_encoder(X_data_set_binary)

  X_data_set_categorical = data_set[['Gender', 'Dependents', 'Property_Area']]
  X_data_set_categorical = apply_imputer_most_frequent(X_data_set_categorical)
  X_data_set_categorical = apply_one_hot_encoder(X_data_set_categorical)

  X_data_set_numerical = data_set[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']]
  X_data_set_numerical = apply_imputer_knn(X_data_set_numerical)
  X_data_set_numerical = apply_standardization(X_data_set_numerical)

  X_data_set = pd.concat([
      pd.DataFrame(X_data_set_binary),
      pd.DataFrame(X_data_set_categorical),
      pd.DataFrame(X_data_set_numerical)
  ], axis=1)

  y_data_set = None

  if 'Loan_Status' in data_set:
    y_data_set = data_set[['Loan_Status']]
    y_data_set = apply_ordinal_encoder(y_data_set)
    y_data_set = pd.DataFrame(y_data_set)

  return X_data_set, y_data_set

## Train and evaluate the models

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold, cross_validate

models = [
    ('Decision Tree', DecisionTreeClassifier()),
    ('kNN', KNeighborsClassifier()),
    ('Naïve Bayes', GaussianNB()),
    ('Support Vector Machine', SVC()),
    ('Neural Network', MLPClassifier())
]
results = {}

X_train, y_train = preprocess_home_loans_data_set(data_set_train)
X_test, y_test = preprocess_home_loans_data_set(data_set_test)

for name, model in models:
        kfold = KFold(n_splits=5, shuffle=True, random_state=90210)
        cv_results = cross_validate(model, X_train, y_train, cv=kfold, scoring=['accuracy'])

        clf = model.fit(X_train, y_train)
        y_pred = clf.predict(X_test)

        results[name] = cv_results['test_accuracy']

/usr/local/lib/python3.9/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.9/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.9/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.9/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exam

1. Decision Tree:
- It is a non-parametric classifier.
- It is simple and interpretable.
- It can handle categorical attributes without preprocessing. (Note: We also performed preprocessing for the categorical attributes in this Colab, which could be one of the reasons why this model ended up having the worst accuracy.)
- It can easily overfit.

2. kNN:
- It is a non-parametric classifier.
- It measures the distance between the attributes, so feature scaling is a must.

3. Naïve Bayes:
- It is a probabilistic classifier.
- It assumes independence between features.
- It may suffer from the "zero-frequency" problem if a feature does not appear in the training data.

4. Support Vector Machine:
- It can handle non-linear decision boundaries.
- It can be sensitive to the choice of kernel function and its parameters.

5. Neural Network:
- It is a versatile classifier that can learn complex patterns in data.
- It may suffer from overfitting if the network is too large or the training data is insufficient.


In [ ]:
pd.DataFrame(results)

,Decision Tree,kNN,Naïve Bayes,Support Vector Machine,Neural Network
0,0.682927,0.715447,0.788618,0.780488,0.772358
1,0.707317,0.747967,0.796748,0.796748,0.772358
2,0.788618,0.788618,0.845528,0.869919,0.813008
3,0.731707,0.707317,0.804878,0.796748,0.821138
4,0.737705,0.762295,0.778689,0.778689,0.762295
